In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Basic Data Cleaning

In [4]:
# final_ems_data_slice columns
columns = [
    "index", "ID", "Hospital_Code", "Date",
    "APOT", "Impression", "Postal_Code", "AgencyNumber",
    "Agency_Unit", "Lat", "Long", "Status",
    
]

target = ["Status"]

In [5]:
# Load the data
file_path = Path('final_ems_data_slice_test.csv')
df = pd.read_csv(file_path, encoding='utf-8')

df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Dropping the APOT column as it is represented by the Status column

df = df.drop(["APOT"], axis=1)
df.head()


,index,ID,Hospital_Code,Date,Impression,Postal_Code,AgencyNumber,Agency_Unit,Lat,Long,Status
0,0,776,508,7/19/2017,T14.90,95820,92905,92905-M22,38.554938,-121.456751,0 (< 20 min)
1,1,824,508,8/20/2017,G89.1,95831,92905,92905-M17,38.554938,-121.456751,0 (< 20 min)
2,2,854,197,9/8/2017,T14.90,95608,92905,92905-M17,38.601860,-121.391908,0 (< 20 min)
3,3,867,475,9/14/2017,R53.1,95624,92905,92905-M23,38.570890,-121.469532,1 (21 - 60 min)
4,4,951,475,10/19/2017,R10.84,95816,92905,92905-M17,38.570890,-121.469532,0 (< 20 min)


In [6]:
# get data types
df.dtypes

index              int64
ID                 int64
Hospital_Code      int64
Date              object
Impression        object
Postal_Code       object
AgencyNumber       int64
Agency_Unit       object
Lat              float64
Long             float64
Status            object
dtype: object

In [7]:
# Creating buckets. The benchmark bucket includes values up to 60 minutes. The extreme buck includes values 61 minutes or more.
# Convert the target column values to benchmark and extreme based on their values
# We can change the names later if needed

x = dict.fromkeys(['0 (< 20 min)', '1 (21 - 60 min)'], 'benchmark')    
df = df.replace(x)

x = dict.fromkeys(['2 (61 - 120 min)', '3 (121 - 180 min)', '4 (181 + min)'], 'extreme')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,index,ID,Hospital_Code,Date,Impression,Postal_Code,AgencyNumber,Agency_Unit,Lat,Long,Status
0,0,776,508,7/19/2017,T14.90,95820,92905,92905-M22,38.554938,-121.456751,benchmark
1,1,824,508,8/20/2017,G89.1,95831,92905,92905-M17,38.554938,-121.456751,benchmark
2,2,854,197,9/8/2017,T14.90,95608,92905,92905-M17,38.601860,-121.391908,benchmark
3,3,867,475,9/14/2017,R53.1,95624,92905,92905-M23,38.570890,-121.469532,benchmark
4,4,951,475,10/19/2017,R10.84,95816,92905,92905-M17,38.570890,-121.469532,benchmark


# Split the Data into Training and Testing

In [8]:
# Create our features
X = df.drop(columns='Status')
X = pd.get_dummies(X)

# Create our target
y = df[target]

In [9]:
X.describe()

,index,ID,Hospital_Code,AgencyNumber,Lat,Long,Date_1/1/2018,Date_1/10/2018,Date_1/11/2018,Date_1/12/2018,...,Agency_Unit_605-S-753,Agency_Unit_605-S-754,Agency_Unit_605-S-755,Agency_Unit_92905-M102,Agency_Unit_92905-M17,Agency_Unit_92905-M2,Agency_Unit_92905-M21,Agency_Unit_92905-M22,Agency_Unit_92905-M23,Agency_Unit_92905-M3
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,...,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,101355.360000,361.748000,7640.800000,223.159364,-119.991811,0.001000,0.005000,0.005000,0.002000,...,0.066000,0.006000,0.050000,0.009000,0.026000,0.002000,0.013000,0.002000,0.021000,0.003000
std,288.819436,44261.754278,147.587345,24466.173628,1937.984056,15.118604,0.031623,0.070569,0.070569,0.044699,...,0.248406,0.077266,0.218054,0.094488,0.159215,0.044699,0.113331,0.044699,0.143456,0.054717
min,0.000000,776.000000,5.000000,605.000000,38.463174,-121.469532,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,249.750000,71723.250000,280.000000,605.000000,38.554938,-121.469532,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,499.500000,106003.500000,475.000000,605.000000,38.570890,-121.456751,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,749.250000,143083.250000,475.000000,605.000000,38.570890,-121.416364,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,999.000000,152172.000000,549.000000,92905.000000,20549.000000,38.573060,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
# Check the balance of our target values
y['Status'].value_counts()

benchmark    902
extreme       98
Name: Status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

# Splitting into Train and Test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(750, 465)

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)



BalancedRandomForestClassifier(random_state=1)

In [13]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6566091954022988

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)


array([[150,  82],
       [  6,  12]], dtype=int64)

In [15]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  benchmark       0.96      0.65      0.67      0.77      0.66      0.43       232
    extreme       0.13      0.67      0.65      0.21      0.66      0.43        18

avg / total       0.90      0.65      0.67      0.73      0.66      0.43       250



In [16]:
# List the features sorted in descending order by feature importance
importances = model.feature_importances_
sorted(zip(model.feature_importances_, X.columns), reverse=True)

[(0.08962291594949229, 'ID'),
 (0.08952634485796647, 'index'),
 (0.056037608947092636, 'Lat'),
 (0.0494964383139922, 'Hospital_Code'),
 (0.04584340078978468, 'Long'),
 (0.02598604082336858, 'Postal_Code_95758'),
 (0.020672400302516526, 'Agency_Unit_605-S-742'),
 (0.019123499632667205, 'Impression_R53.1'),
 (0.017669563277188875, 'Agency_Unit_605-S-746'),
 (0.017074497001956332, 'Postal_Code_95608'),
 (0.014964294435672211, 'Impression_F99'),
 (0.014597383812123901, 'Agency_Unit_605-S-751'),
 (0.012623848820516544, 'Impression_R10.84'),
 (0.012497546590876525, 'Postal_Code_95816'),
 (0.011588943039697102, 'Impression_I20.9'),
 (0.0113194054459279, 'Postal_Code_95823'),
 (0.011318173144558819, 'Postal_Code_95838'),
 (0.01104729210892314, 'Impression_M79.60'),
 (0.010223281609293646, 'Agency_Unit_605-S-753'),
 (0.009658045131190984, 'Impression_T14.90'),
 (0.009368576559504965, 'Date_9/11/2018'),
 (0.009147038647204557, 'Impression_J98.01'),
 (0.009100711159704448, 'Impression_R41.82'),
 

### Easy Ensemble AdaBoost Classifier

In [17]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [18]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6182950191570882

In [19]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[158,  74],
       [  8,  10]], dtype=int64)

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  benchmark       0.95      0.68      0.56      0.79      0.62      0.38       232
    extreme       0.12      0.56      0.68      0.20      0.62      0.37        18

avg / total       0.89      0.67      0.56      0.75      0.62      0.38       250

